## TextRank

In [3]:
import pandas as pd
from summa import keywords

# CSV 파일 읽기
file_path = "news_data_detailed.csv"  # CSV 파일 경로
df = pd.read_csv(file_path)

# 칼럼명 가져오기
columns = df.columns
print("칼럼명:", columns.tolist())  # 리스트 형태로 출력

# 전체 행 수 확인
total_rows = len(df)
print(f"CSV 파일의 총 행 수: {total_rows}")

texts = df["뉴스 내용"].dropna().tolist()

칼럼명: ['뉴스 제목', '뉴스 내용', '뉴스 반응 수', '신문사 이름', '기자 이름', '기자의 구독자 수', '기자의 응원해요 수']
CSV 파일의 총 행 수: 100


In [4]:
textrank_keywords = [keywords.keywords(text, words=10, split=True) for text in texts]
textrank_keywords

[['올해', '대해', '김재관', '일', '문화를', '최고의', '위한 소통', '세션에서는', '지난해'],
 ['밸류업', '강조했다', '우리금융을', '이어', '채종원', '우리금융그룹 회장이 해외 투자자들에게'],
 ['등', '트럼프 취임', '것으로', '전', '조치를', '일', '행정명령', '미국 대통령'],
 ['가장', '상품', '여행', '투어', '나타났다', '클룩', '플랫폼', 'kkday가', '수도권', '다양한'],
 ['있다', '수', '직접', '있는', '한맥', '오비맥주', '다양한', '거품을'],
 ['ai', '있다', '글로벌', '등', '수', 'llm', '미국', '빅테크', '모델'],
 ['대미 수출', '트럼프', '미국', '관세를', '년', '것으로', '지난해', '억', '투자'],
 ['설', '협력사들에게', '개', '중', '협력과 경영 안정을 위한 지원을 이어가고'],
 ['한남', '구역', '조합원', '삼성물산은', '이에', '삼성물산', '한남뉴타운', '재개발', '이번', '용산구'],
 ['미국', '올해', '것으로', '있다', '주요', '시장이', 'fomc', '때문에'],
 ['일', '비트코인', '최고가를', '억', '역대', '강남구 업비트 고객센터', '비트코인이'],
 ['건', '시장', '이달', '투자주의 종목에 지정된', '경우', '지정됐다', '상장폐지', '주가가'],
 ['카드', '서울', 'photocdj', '있다', '참가자들이 배틀을 펼치고', '코리아 오픈'],
 ['카드', '서울', 'photocdj', '있다', '참가자들이 배틀을 펼치고', '코리아 오픈'],
 ['기반시설을', '현금', '기', '재건축', '공공기여금 유동화', '공공기여금을', '있다', '조합이'],
 ['한남', '일대', '서울', '용산구', 'saba', '조 육박할 정도의', '약'],
 ['가격이', '수입', 

## TextRank + TF - IDF

### TextRank와 TF-IDF 사용했을 때의 장점
A. 품질 개선  
TF-IDF로 빈도 기반의 중요한 단어를 선택하고, TextRank로 문맥적으로 중요한 단어를 보완하여 더 정확한 키워드 추출이 가능.  
B. 효율성 증가  
TF-IDF로 사전 후보를 줄여, TextRank의 계산량을 감소시킬 수 있음.  
C. 다양한 텍스트 처리 가능  
긴 문서와 짧은 문서 모두에서 균형 잡힌 결과 제공.  

In [12]:
# import requests

# # 한국어 불용어 리스트 가져오기
# url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ko/master/stopwords-ko.txt"
# response = requests.get(url)
# stop_words = response.text.split("\n")

# print("불용어 리스트:", stop_words[:50])  # 상위 50개 불용어 출력

# 불용어 직접 정의
korean_stopwords = [
    # 조사
    "은", "는", "이", "가", "을", "를", "의", "에", "에서", "에게", "로", "으로", "와", "과",
    # 어미
    "하다", "이다", "합니다", "했다", "되다", "한다", "됐다", "있다", 
    
    # 접속사
    "그리고", "하지만", "그러나", "또한", "따라서", "그러므로", "왜냐하면", "따르면"
]

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. TF-IDF 벡터라이저 설정
vectorizer = TfidfVectorizer(
    stop_words = korean_stopwords,  # 불용어 제거
    max_features=20,      # 상위 20개의 키워드 추출
)

# 2. 텍스트 데이터를 TF-IDF로 변환
tfidf_matrix = vectorizer.fit_transform(texts)

# 3. 키워드와 TF-IDF 점수 매핑
feature_names = vectorizer.get_feature_names_out()

# 4. 각 문서별 키워드 추출
tfidf_keywords = []
for i, text in enumerate(texts):
    # 각 단어의 TF-IDF 점수
    tfidf_scores = tfidf_matrix[i].toarray().flatten()
    sorted_indices = tfidf_scores.argsort()[::-1]  # 점수 내림차순 정렬
    tfidf_keyword = [(feature_names[idx]) for idx in sorted_indices if tfidf_scores[idx] > 0]
    tfidf_keywords.append(tfidf_keyword)
tfidf_keywords

[['올해', '지난', '지난해', '대한', '밝혔다', '통해', '이라고', '19일'],
 ['말했다', '기자', '이라고', '19일'],
 ['트럼프', '미국', '것으로', '주요', '밝혔다', '통해', '있는', '이라고'],
 ['것으로', '평균', '국내', '주요', '대한', '올해', '말했다', '있는', '이라고'],
 ['있는', '주요', '올해', '통해', '19일'],
 ['미국', '국내', '통해', '있는', '지난해', '주요', '대한', '말했다', '이라고', '대비', '것으로', '지난'],
 ['트럼프', '미국', '지난해', '것으로', '주요', '대한', '지난', '있는'],
 ['밝혔다', '통해', '19일'],
 ['서울', '주요', '올해', '말했다', '지난해', '것으로', '19일'],
 ['미국', '올해', '주요', '것으로', '트럼프', '통해', '말했다', '대비', '대한', '기자'],
 ['트럼프', '미국', '국내', '서울', '지난해', '19일'],
 ['대한', '국내', '말했다', '통해', '있는', '지난해', '것으로', '19일'],
 ['서울', '기자', '19일'],
 ['서울', '기자', '19일'],
 ['것으로', '자금', '국내', '통해', '19일'],
 ['서울', '기자', '19일'],
 ['국내', '주요', '기자', '있는', '지난해'],
 ['서울', '대비', '지난', '기자', '19일'],
 ['지난', '말했다', '통해', '이라고', '국내', '있는', '지난해', '19일'],
 ['평균', '올해', '기자', '있는'],
 ['서울', '기자', '19일'],
 ['자금', '평균', '올해', '대비', '밝혔다', '것으로', '서울', '기자', '있는', '19일'],
 ['대한', '지난', '말했다', '것으로', '국내', '밝혔다', '지난해', '19일'],
 ['

In [41]:
# 3. TF-IDF와 TextRank 결합

# 교집합 추출
combined_keywords = []
for i in range(len(texts)):
    combined_keyword = set(tfidf_keywords[i]).intersection(set(textrank_keywords[i]))
    combined_keywords.append(combined_keyword)
    
combined_keywords

[{'올해', '지난해'},
 set(),
 {'것으로'},
 set(),
 {'있는'},
 {'미국'},
 {'것으로', '미국', '지난해', '트럼프'},
 set(),
 set(),
 {'것으로', '미국', '올해', '주요'},
 set(),
 set(),
 {'서울'},
 {'서울'},
 set(),
 {'서울'},
 set(),
 {'서울'},
 set(),
 set(),
 {'서울'},
 {'올해', '평균'},
 {'지난'},
 {'것으로', '서울', '지난해'},
 {'지난해'},
 {'것으로', '서울'},
 {'주요', '트럼프'},
 {'통해'},
 set(),
 set(),
 {'서울'},
 set(),
 {'지난해'},
 set(),
 set(),
 {'대비', '평균'},
 {'올해', '자금', '평균'},
 {'자금', '평균'},
 {'지난해'},
 set(),
 set(),
 set(),
 {'것으로', '국내', '대비', '지난해'},
 {'것으로', '지난해'},
 {'주요', '지난해'},
 {'자금'},
 {'미국', '트럼프'},
 {'국내'},
 {'서울'},
 {'미국', '트럼프'},
 set(),
 set(),
 {'국내', '트럼프'},
 set(),
 {'것으로'},
 set(),
 set(),
 set(),
 set(),
 set(),
 {'통해'},
 {'대한'},
 set(),
 {'트럼프'},
 {'지난해'},
 {'자금', '평균'},
 set(),
 {'대한'},
 {'미국'},
 {'통해'},
 {'지난해'},
 {'있는'},
 {'기자'},
 set(),
 {'트럼프'},
 {'서울'},
 set(),
 {'것으로', '트럼프'},
 {'있는'},
 {'국내', '대비'},
 set(),
 {'지난해'},
 set(),
 {'서울'},
 {'지난해'},
 set(),
 set(),
 set(),
 set(),
 {'미국', '주요', '트럼프'},
 {'자금', '평균'},
 {'트럼프

## konlpy의 Okt를 사용해서 형태소 제거

In [11]:
from summa import keywords
from konlpy.tag import Okt

# 원문 텍스트
text = """
재테크는 초보자에게도 중요한 경제 관리 방법입니다. 금테크와 주식 투자 등 다양한 전략이 있습니다.
다이어트와 운동은 건강 관리에 필수적입니다. 유산소 운동은 체중 감량에 효과적입니다.
인공지능(AI)은 기술 발전의 핵심입니다. AI를 활용한 자동화 시스템은 생산성을 높이고 있습니다.
"""

# 1. TextRank로 키워드 추출
textrank_keywords = keywords.keywords(text, words=10, split=True)  # 상위 10개 키워드
print("TextRank 키워드:", textrank_keywords)

# 2. Okt 형태소 분석기
okt = Okt()

# 3. 형태소 분석 후 정제 함수
def refine_keywords(keywords):
    refined = []
    for word in keywords:
        # 형태소 분석 수행
        pos = okt.pos(word)
        for w, p in pos:
            if p in ["Noun", "Verb"]:  # 명사와 동사만 포함
                refined.append(w)
    return list(set(refined))  # 중복 제거 후 반환

# 4. TextRank 키워드 정제
refined_keywords = refine_keywords(textrank_keywords)
print("정제된 키워드:", refined_keywords)

TextRank 키워드: ['운동은', '있습니다', '초보자에게도 중요한 경제 관리', '방법입니다 금테크와 주식', '발전의']
정제된 키워드: ['금', '운동', '방법', '주식', '경제', '발전', '초보자', '관리', '테크']
